In [ ]:
ls

In [ ]:
pwd

In [ ]:
ls /

In [ ]:
ls /datasets

In [ ]:
#!curl -o /datasets/tl_2024_us_state.zip https://www2.census.gov/geo/tiger/TIGER2024/STATE/tl_2024_us_state.zip

In [ ]:
ls /datasets/tl_2024_us_state.zip

In [ ]:
#! unzip /datasets/tl_2024_us_state.zip -d /datasets/tl_2024_us_state

In [ ]:
ls /datasets/tl_2024_us_state

# Geopandas DC geodataframe slice from Census shp files

In [ ]:
import geopandas as gpd

In [ ]:
gdf_us = gpd.read_file('/datasets/tl_2024_us_state/tl_2024_us_state.shp')

In [ ]:
gdf_us.head(2)

In [ ]:
gdf_us.columns

In [ ]:
gdf_us.items

In [ ]:
gdf_us.iloc[53]

In [ ]:
gdf_dc = gdf_us[gdf_us.STUSPS=='DC']

In [ ]:
gdf_dc

In [ ]:
gdf_dc.crs

In [ ]:
gdf_dc.bounds

In [ ]:
gdf_dc.plot()

In [ ]:
type(gdf_dc.bounds.values)

In [ ]:
gdf_dc.bounds.values.shape

In [ ]:
gdf_dc.bounds.values

In [ ]:
gdf_dc.bounds.values.tolist()

In [ ]:
# index into (1,4) shape to get inner array len 4
list_dc_bounds = gdf_dc.bounds.values.tolist()[0]

#### Python list of dc boundary coordinates

In [ ]:
list_dc_bounds

## python dictionary of parameters for stac search

In [ ]:
params = {}

In [ ]:
params['limit'] = 400

# tuple is immutable, represent coordinates as tuple and not list - which is mutable
params['bbox'] = tuple(list_dc_bounds)

date_time = "2022-01-01T00:00:00Z/2022-12-31T23:59:59Z"
params['datetime'] = date_time

collections = ['landsat-c2l2-sr']
params['collections'] = collections

params['query'] = {"platform": {"in": ["LANDSAT_9"]},
                   'eo:cloud_cover':{'lte': 20}}

In [ ]:
params

## requests to call stac with parameters dictionary

In [ ]:
import requests as rq

In [ ]:
def fetch_stac_server(query):
    '''
    Queries the stac-server (STAC) backend.
    query is a python dictionary to pass as json to the request.
    '''
    
    search_url = f"https://landsatlook.usgs.gov/stac-server/search"
    query_return = rq.post(search_url, json=query).json()
    error = query_return.get("message", "")
    if error:
        raise Exception(f"STAC-Server failed and returned: {error}")
        
    print(f"Items Found: {len(query_return['features'])}")    
    
    for q in query_return['features']: print(f"Platform: {q['properties']['platform']}, Cloud Cover: {q['properties']['eo:cloud_cover']}, Collection: {q['description']}, ID: {q['id']}")
        
    return query_return

In [ ]:
query_return = fetch_stac_server(params)

In [ ]:
query_return

In [ ]:
len(query_return)

In [ ]:
type(query_return)

In [ ]:
query_return['features']

In [ ]:
params

In [ ]:
def list_item_assets(query_item):
    print(query_item['id'])
    
    # Print some of the  metadata attributes for the product
    print(f"> Product Family: {query_item['description']}")
    print(f"> Product ID: {query_item['id']}")
    print(f"> Acquisition Date: {query_item['properties']['datetime']}")
    print(f"> Spacecraft (Platform): {query_item['properties']['platform']}")
    print(f"> Cloud Cover: {query_item['properties']['eo:cloud_cover']}%.")
    print(f"> Number of assets: {len(query_item['assets'])} assets")
    print(f"> BBOX: {query_item['bbox']}") 
    
    for a in query_item['assets']: #print assets in each feature
        
        print('\n'+ a)
        
        try:
            print(f"Asset: {query_item['assets'][a]['title']}")
            print(f"type: {query_item['assets'][a]['type']}")
            print(f"Description: {query_item['assets'][a]['description']}")
            print(f"Role: {query_item['assets'][a]['roles']}")
            print(f"S3 URL: {query_item['assets'][a]['alternate']['s3']['href']}")
            
            # for asset in query_item['assets']:
            #     print(f"{asset}: {query_item['assets'][asset]['title']}")

        except:
            print(f"Role: {query_item['assets'][a]['roles']}")
            print(f"URL: {query_item['assets'][a]['href']}")

            continue
            
    return 0

In [ ]:
for feature in query_return['features']:
    print(feature['assets']['blue'])
    print(feature['assets']['green'])
    print(feature['assets']['red'])
    print()

In [ ]:
query_item = query_return['features'][0] #search a single feature (item) in the collection

In [ ]:
type(query_item)

In [ ]:
#query_item.keys()
[print(key) for key in query_item.keys()]

In [ ]:
query_item

In [ ]:
list_item_assets(query_item)

In [ ]:
query_item['assets'].keys()

In [ ]:
for el in list(query_item['assets'].keys()):
    print(el)
    print()
    print(query_item['assets'][el])
    print()

In [ ]:
bands = ['red', 'green', 'blue']
band_ids = []
band_links = []

In [ ]:
for b in bands: #sort through a string to find relevant band names
        band_links.append(query_item['assets'][b]['alternate']['s3']['href'])
        band_ids.append(query_item['assets'][b]['eo:bands'][0]['name'])

In [ ]:
print(bands,'\n')
print(band_links,'\n')
print(band_ids,'\n')

## request to stac sever with metadata to locate specific cog

In [ ]:
# pip install rasterio
import requests
import rasterio as rio

In [ ]:
# url = 's3://usgs-landsat/collection02/level-2/standard/oli-tirs/2024/015/033/LC09_L2SP_015033_20241020_20241022_02_T1/LC09_L2SP_015033_20241020_20241022_02_T1_SR_B4.TIF'

url = band_links[1]

In [ ]:
print(url)

In [ ]:
local_file = 'DC_4.tif'

In [ ]:
ls

## download cog

In [ ]:
def retrieve_cog(geotiff_path, aoi_geodf):
    """
    Retrieve a Cloud-Optimized GeoTIFF (COG) within a specified bounding box.

    Parameters:
        - geotiff_path (str): Path to the COG GeoTIFF file.
        - aoi_geodf (geopandas.GeoDataFrame): GeoDataFrame representing the area of interest.

    Returns:
        - cog (numpy.ndarray): Numpy array containing the data within the specified bounding box.
    """
    with rio.Env(aws_session):
        with rio.open(geotiff_path) as src:
            # Assuming aoi_geodf has a different CRS, transform it to match the CRS of the raster
            aoi_geodf = aoi_geodf.to_crs(src.crs)
            cog = src.read(1, window=from_bounds(
                aoi_geodf.bounds.minx.iloc[0],
                aoi_geodf.bounds.miny.iloc[0],
                aoi_geodf.bounds.maxx.iloc[0],
                aoi_geodf.bounds.maxy.iloc[0],
                src.transform))
    return cog

In [ ]:
bbox_in =[
            gdf_dc.bounds.minx.iloc[0],
            gdf_dc.bounds.miny.iloc[0],
            gdf_dc.bounds.maxx.iloc[0],
            gdf_dc.bounds.maxy.iloc[0],
]

In [ ]:
bbox_in

In [ ]:
xmin, ymin, xmax, ymax = bbox_in #list_dc_bounds
bbox_in = [[ymin, xmin], [ymax, xmax]]
center_in = [(ymin + ymax) / 2, (xmin + xmax) / 2]

In [ ]:
gdf_dc.bounds.minx.iloc[0]

In [ ]:
gdf_4326 = gdf_dc.to_crs(epsg=4326)
gdf_4326.crs

In [ ]:
geotiff_path_in = 's3://usgs-landsat/collection02/level-2/standard/oli-tirs/2022/015/033/LC09_L2SP_015033_20221202_20230319_02_T1/LC09_L2SP_015033_20221202_20230319_02_T1_SR_B4.TIF'
#gdf_dc

import boto3
from rasterio.session import AWSSession
from rasterio.windows import from_bounds

aws_session = AWSSession(boto3.Session(), requester_pays=True)
cog_in = retrieve_cog(geotiff_path_in, gdf_4326)

In [ ]:
cog_in

In [ ]:
from PIL import Image
Image.fromarray(cog_in)

In [ ]:
# Folium map and overlay
m = folium.Map(location=center_in, zoom_start=12)

folium.raster_layers.ImageOverlay(
    image=cog_in,
    bounds=bbox_in,
    opacity=0.9,
    interactive=True,
    cross_origin=False
).add_to(m)

In [ ]:
m

In [ ]:
#!pip install boto3

In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'  # or your preferred region

In [ ]:
import boto3

s3 = boto3.client('s3')
print(s3.list_buckets())

In [ ]:
#!pip install boto3
import boto3
from rasterio.session import AWSSession

# session = boto3.Session()
# aws_session = AWSSession(session)
aws_session = AWSSession(boto3.Session(), requester_pays=True)

In [ ]:
aws_session

In [ ]:
pwd

In [ ]:
import rasterio as rio
from rasterio.session import AWSSession
import boto3
from rasterio.env import Env

# Step 1: Start an AWS session using local credentials (must be valid)
session = boto3.Session()
aws_session = AWSSession(session, requester_pays=True)

# Step 2: Config required for requester pays buckets
config = {
    'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
    'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.TIF',
    'AWS_REQUEST_PAYER': 'requester'
}

# Step 3: URL of the S3 object
url = 's3://usgs-landsat/collection02/level-2/standard/oli-tirs/2024/015/033/LC09_L2SP_015033_20241020_20241022_02_T1/LC09_L2SP_015033_20241020_20241022_02_T1_SR_B4.TIF'

# Step 4: Open with rasterio using the session and config
with Env(session=aws_session, **config):
    with rio.open(url) as src:
        cog = src.read(1)
        print("Shape:", cog.shape)
        print("Metadata:", src.meta)


In [ ]:
url='s3://usgs-landsat/collection02/level-2/standard/oli-tirs/2024/015/033/LC09_L2SP_015033_20241020_20241022_02_T1/LC09_L2SP_015033_20241020_20241022_02_T1_SR_B4.TIF'


In [ ]:
url

In [ ]:
# url = '/vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF'
# url='s3://usgs-landsat/collection02/level-2/standard/oli-tirs/2024/015/033/LC09_L2SP_015033_20241020_20241022_02_T1/LC09_L2SP_015033_20241020_20241022_02_T1_SR_B4.TIF'

#AWS_NO_SIGN_REQUEST='YES'
#with Env(session=aws_session, **config):
with Env(**config, session=aws_session, AWS_NO_SIGN_REQUEST='YES'):
    with rio.open(url) as src:
        cog = src.read(1)
        print(src)


In [ ]:
import rasterio as rio
from rasterio.env import Env

url='s3://usgs-landsat/collection02/level-2/standard/oli-tirs/2024/015/033/LC09_L2SP_015033_20241020_20241022_02_T1/LC09_L2SP_015033_20241020_20241022_02_T1_SR_B4.TIF'

config = {
    'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
    'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.TIF',
    'GDAL_HTTP_MAX_RETRY': '10',
    'GDAL_HTTP_RETRY_DELAY': '5',
    'GDAL_HTTP_TIMEOUT': '30'
}


#AWS_NO_SIGN_REQUEST='YES'
with Env(**config):
    with rio.open(url) as src:
        
        print(src.crs)
        bounds = src.bounds  # returns (left, bottom, right, top)
        cog = src.read(1, masked=True)  # `masked=True` can handle partial reads better

In [ ]:
cog

In [ ]:
type(cog)

In [ ]:
cog.ndim

In [ ]:
cog.shape

In [ ]:
cog.dtype

In [ ]:
cog[3333][3333]

In [ ]:
cog.crs

In [ ]:
del cog

In [ ]:
cog

In [ ]:
type(Image.fromarray(cog))

In [ ]:
from PIL import Image

Image.fromarray(cog)

In [ ]:
import numpy as np

def array_to_rgba_mask_nodata(array, nodata_value=None):
    # Normalize to 0–255 for display
    arr = array.astype(np.float32)
    arr_min, arr_max = np.nanmin(arr), np.nanmax(arr)
    arr = 255 * (arr - arr_min) / (arr_max - arr_min)
    arr = arr.astype(np.uint8)

    # Create RGB (grayscale)
    rgb = np.stack([arr]*3, axis=-1)

    # Alpha: fully opaque
    alpha_channel = np.full_like(arr, 255, dtype=np.uint8)

    # Make NoData pixels transparent
    if nodata_value is not None:
        mask = array == nodata_value
        alpha_channel[mask] = 0

    rgba = np.dstack([rgb, alpha_channel])  # shape: (H, W, 4)
    # return Image.fromarray(rgba)
    return rgba
    

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile
from rasterio.session import AWSSession
import boto3

# Step 1: Authenticated session (for requester pays bucket)
session = boto3.Session()
aws_session = AWSSession(session, requester_pays=True)

# Step 2: S3 COG URL (EPSG:32618)
url = 's3://usgs-landsat/collection02/level-2/standard/oli-tirs/2024/015/033/LC09_L2SP_015033_20241020_20241022_02_T1/LC09_L2SP_015033_20241020_20241022_02_T1_SR_B4.TIF'

# Step 3: Desired destination CRS
dst_crs = 'EPSG:4326'

# Step 4: Open source COG from S3 and reproject in memory
with rasterio.Env(session=aws_session, AWS_REQUEST_PAYER='requester'):
    with rasterio.open(url) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)

        dst_meta = src.meta.copy()
        dst_meta.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Step 5: Use MemoryFile to hold the reprojected image
        with MemoryFile() as memfile:
            with memfile.open(**dst_meta) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest
                    )

                # Read back the reprojected data
                img = dst.read(1)
                print("Reprojected array shape:", img.shape)
                result_array = array_to_rgba_mask_nodata(img, nodata_value=0)  # or whatever your nodata is


                bounds = dst.bounds  # returns (left, bottom, right, top)

In [ ]:
result_array

In [ ]:
Image.fromarray(result_array)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import folium
from io import BytesIO
import base64

In [ ]:
# xmin, ymin, xmax, ymax = list_dc_bounds
xmin, ymin, xmax, ymax = bounds #list_dc_bounds
bbox = [[ymin, xmin], [ymax, xmax]]
center = [(ymin + ymax) / 2, (xmin + xmax) / 2]

In [ ]:
bbox

In [ ]:
bounds

In [ ]:
center

In [ ]:
# # Save masked image to an in-memory PNG
# buf = BytesIO()
# plt.imsave(buf, cog, format='png', cmap='gray')  # or 'viridis', etc.
# buf.seek(0)

from PIL import Image
img_pil = Image.fromarray(result_array)
# Convert PIL image to base64-encoded PNG
buf = BytesIO()
img_pil.save(buf, format='PNG')
img_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
img_url = f"data:image/png;base64,{img_base64}"


In [ ]:
#plt.imshow()

In [ ]:
# # Encode image as base64 for embedding
# img_base64 = base64.b64encode(buf.read()).decode('utf-8')
# img_url = f"data:image/png;base64,{img_base64}"


In [ ]:
#img_url is large

import sys
sys.getsizeof(img_url)

In [ ]:
# Folium map and overlay
m = folium.Map(location=center, zoom_start=12)

In [ ]:
folium.raster_layers.ImageOverlay(
    image=img_url,
    bounds=bbox,
    opacity=0.9,
    interactive=True,
    cross_origin=False
).add_to(m)

In [ ]:
folium.Rectangle(bounds=bbox, color="purple", fill=False).add_to(m)
folium.LayerControl().add_to(m)

In [ ]:
m  # Display in Jupyter


In [ ]:
layer_count = len(m._children)
layer_count

In [ ]:
print(f"Total layers (including controls, overlays, tiles): {layer_count}")

In [ ]:
overlay_names = [child.layer_name for child in m._children.values()
                 if hasattr(child, 'layer_name')]
print("Named overlays:", overlay_names)

In [ ]:
folium.LayerControl().add_to(m)


In [ ]:
folium.LayerControl(collapsed=False).add_to(m)


In [ ]:
m

In [ ]:
del m

In [ ]:
#!pip install folium
!pip install matplotlib


In [ ]:
import rasterio as rio
from rasterio.plot import reshape_as_image
from rasterio.env import Env
from rasterio.warp import transform_bounds
import folium
import matplotlib.pyplot as plt
import numpy as np
import tempfile
import os

# ✅ Public COG URL from Google Cloud (same image as your original)
url_new = "/vsicurl/https://storage.googleapis.com/gcp-public-data-landsat/LC09/01/015/033/LC09_L2SP_015033_20241020_20241022_02_T1/LC09_L2SP_015033_20241020_20241022_02_T1_SR_B4.TIF"

# ✅ GDAL config for remote access
config = {
    'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
    'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.TIF',
    'GDAL_HTTP_MAX_RETRY': '10',
    'GDAL_HTTP_RETRY_DELAY': '5',
    'GDAL_HTTP_TIMEOUT': '30'
}

# ✅ Open and process image
with Env(**config):
    with rio.open(url_new) as src:
        # Read band 4
        img = src.read(1, masked=True)
        bounds = src.bounds
        crs = src.crs

        # Reproject bounds to WGS84 (lat/lon) for folium
        bounds_wgs84 = transform_bounds(crs, 'EPSG:4326', *bounds)
        bbox = [
            [bounds_wgs84[1], bounds_wgs84[0]],  # [south, west]
            [bounds_wgs84[3], bounds_wgs84[2]]   # [north, east]
        ]

        # Save preview image
        temp_file = os.path.join(tempfile.gettempdir(), 'landsat_b4_preview.png')
        plt.imsave(temp_file, img, cmap='gray')

        # Compute center of image
        center_lat = (bounds_wgs84[1] + bounds_wgs84[3]) / 2
        center_lon = (bounds_wgs84[0] + bounds_wgs84[2]) / 2

# ✅ Plot on folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add grayscale image overlay
folium.raster_layers.ImageOverlay(
    name="Landsat Band 4",
    image=temp_file,
    bounds=bbox,
    opacity=0.6,
    interactive=True,
    cross_origin=False
).add_to(m)

# Add bounding box
folium.Rectangle(
    bounds=bbox,
    color='red',
    weight=2,
    fill=False,
    dash_array='5,5',
    tooltip='COG bounds'
).add_to(m)

folium.LayerControl().add_to(m)
m  # Show map


In [ ]:
config

In [ ]:
cog_url

In [ ]:
with Env(session=aws_session, **config):
    with rio.open(cog_url) as src:
        cog = src.read(1)

In [ ]:
Image.fromarray(cog)

In [ ]:
#!pip install boto3
import boto3
from rasterio.session import AWSSession

# session = boto3.Session()
# aws_session = AWSSession(session)
aws_session = AWSSession(boto3.Session(), requester_pays=True)



from rasterio.plot import show as rio_show

rio_show(retrieve_cog(band_links[0],list_dc_bounds), cmap='viridis')

In [ ]:
import rasterio as rio
from rasterio.env import Env

cog_url = '/vsicurl/https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF'

config = {
    'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
    'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.TIF',
    'GDAL_HTTP_MAX_RETRY': '10',
    'GDAL_HTTP_RETRY_DELAY': '5',
    'GDAL_HTTP_TIMEOUT': '30'
}

with Env(**config):
    with rio.open(cog_url) as src:
        cog = src.read(1, masked=True)  # `masked=True` can handle partial reads better

In [ ]:
cog

In [ ]:
del cog

In [ ]:
from PIL import Image

Image.fromarray(cog)

In [ ]:
#Image.fromarray(cog).save("/datasets/SAT_landsat_b4.png")